In [1]:
import pandas as pd
import numpy as np
import re
import csv
from string import ascii_lowercase
import random

In [2]:
subset_punk_df = pd.read_csv('../data/subset_punk_bands.csv')
subset_punk_df.drop('Unnamed: 0', axis = 1, inplace = True)

## Creating a model

### Setting up the data

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Input, Embedding, Dropout, concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Model

In [4]:
# "randomly" choose one band to test if model works
yellowcard = subset_punk_df.LYRICS[subset_punk_df.ARTIST_NAME == 'yellowcard']


# for full model
# take out spaces in the artist names
new_artist_names = [re.sub(r'\W', '', string = subset_punk_df.ARTIST_NAME[w]) for w in range(len(subset_punk_df.ARTIST_NAME))]

tokenizer_artist = Tokenizer()
tokenizer_artist.fit_on_texts(new_artist_names)
artist_seq = tokenizer_artist.texts_to_sequences(new_artist_names)

tokenizer_lyrics = Tokenizer()
tokenizer_lyrics.fit_on_texts([str(lyr) for lyr in subset_punk_df.LYRICS])

token_seq = tokenizer_lyrics.texts_to_sequences([str(lyr) for lyr in subset_punk_df.LYRICS])


n_gram_seq = []
artists = []
# for every line in tokenized sequences
for line, band in zip(token_seq, artist_seq):
    # used to append the token_seq starting from 0th element to 1st element
    for length in range(2, len(line)):
        n_gram_seq.append(line[:length])
        artists.append(band)
        
artists = np.array(artists)

In [5]:
# create padded sequences
n_gram_seq_padded = pad_sequences(n_gram_seq, maxlen = 11)

In [6]:
n_gram_seq_padded[:11]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,  700, 1615],
       [   0,    0,    0,    0,    0,    0,    0,    0,  700, 1615,   17],
       [   0,    0,    0,    0,    0,    0,    0,  700, 1615,   17,    3],
       [   0,    0,    0,    0,    0,    0,  700, 1615,   17,    3, 1452],
       [   0,    0,    0,    0,    0,  700, 1615,   17,    3, 1452,   90],
       [   0,    0,    0,    0,  700, 1615,   17,    3, 1452,   90,   20],
       [   0,    0,    0,  700, 1615,   17,    3, 1452,   90,   20,  155],
       [   0,    0,  700, 1615,   17,    3, 1452,   90,   20,  155,    7],
       [   0,  700, 1615,   17,    3, 1452,   90,   20,  155,    7,  425],
       [ 700, 1615,   17,    3, 1452,   90,   20,  155,    7,  425,    2],
       [1615,   17,    3, 1452,   90,   20,  155,    7,  425,    2,  407]],
      dtype=int32)

In [7]:
subset_punk_df

,ARTIST_NAME,ARTIST_URL,SONG_NAME,SONG_URL,LYRICS
0,all time low,https://www.azlyrics.com/a/alltimelow.html,i can't do the one-two step,https://www.azlyrics.com/lyrics/alltimelow/ica...,front page of the magazine said don't believe ...
1,all time low,https://www.azlyrics.com/a/alltimelow.html,the girl's a straight-up hustler,https://www.azlyrics.com/lyrics/alltimelow/the...,lipstick has a way of leaving more than just a...
2,all time low,https://www.azlyrics.com/a/alltimelow.html,"sticks, stones, and techno",https://www.azlyrics.com/lyrics/alltimelow/sti...,you spin your words like a record in motion st...
3,all time low,https://www.azlyrics.com/a/alltimelow.html,coffee shop soundtrack,https://www.azlyrics.com/lyrics/alltimelow/cof...,should i write myself out of the history books...
4,all time low,https://www.azlyrics.com/a/alltimelow.html,break out! break out!,https://www.azlyrics.com/lyrics/alltimelow/bre...,luck loves me not tonight i'm running out this...
...,...,...,...,...,...
544,yellowcard,https://www.azlyrics.com/y/yellowcard.html,what appears,https://www.azlyrics.com/lyrics/yellowcard/wha...,slow steady hands waving their last goodbye th...
545,yellowcard,https://www.azlyrics.com/y/yellowcard.html,got yours,https://www.azlyrics.com/lyrics/yellowcard/got...,stacking bricks on broken ground building towe...
546,yellowcard,https://www.azlyrics.com/y/yellowcard.html,a place we set afire,https://www.azlyrics.com/lyrics/yellowcard/apl...,you feel it you boxed it by the youth you left...
547,yellowcard,https://www.azlyrics.com/y/yellowcard.html,leave a light on,https://www.azlyrics.com/lyrics/yellowcard/lea...,so where are you and how's it been how's the w...


In [8]:
# create labels by using One Hot Encoding 
labels = to_categorical(n_gram_seq_padded[:,-1:])
X = n_gram_seq_padded[:,:-1]

# training size (0.8 of the model)
train_size = round(n_gram_seq_padded.shape[0]*0.8)
# get randomly chosen indices of artists for training
ids = np.random.choice(range(len(artists)), train_size, replace = False)

# create test and train
y_train = labels[ids]
y_test = np.delete(labels, ids, axis = 0)

lyrics_train = X[ids]
lyrics_test = np.delete(X, ids, axis = 0)

artist_train = artists[ids]
artist_test = np.delete(artists, ids, axis = 0)

# find largest vocab size in padded sequence; this is input size
vocab_size = max([w for sentence in n_gram_seq_padded for w in sentence]) + 1
artist_size = max([len(art) for art in artists]) + 1

In [9]:
# define the two inputs
inputA = Input(shape = (10, ))
inputB = Input(shape = (1,))

# first branch for first input
lyrics = Embedding(input_dim = vocab_size, output_dim = 64, input_length = 10)(inputA)
lyrics = Bidirectional(LSTM(128, return_sequences = True))(lyrics)
lyrics = Dropout(0.2)(lyrics)
lyrics = LSTM(64)(lyrics)
lyrics = Dense(round(vocab_size/2), activation = 'relu')(lyrics)
lyrics = Dense(vocab_size, activation = 'softmax')(lyrics)
lyrics = Model(inputs = inputA, outputs = lyrics)

# second branch for second input
artist = Dense(64, activation = 'relu')(inputB)
artist = Dense(10, activation = 'relu')(artist)
artist = Dense(vocab_size, activation = 'relu')(artist)
artist = Model(inputs = inputB, outputs = artist)


# combine output of branches
combined = concatenate([lyrics.output, artist.output])


z = Dense(2, activation = 'relu')(combined)
z = Dense(vocab_size, activation = 'softmax')(z)
model = keras.Model(inputs = [lyrics.input, artist.input], outputs = z)

#compile model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = 'accuracy')

In [10]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 64)       382656      input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 10, 256)      197632      embedding[0][0]                  
__________________________________________________________________________________________________
dropout (Dropout)               (None, 10, 256)      0           bidirectional[0][0]              
____________________________________________________________________________________________

In [11]:
model.fit([lyrics_train, artist_train], y_train, epochs = 8, validation_data = ([lyrics_test, artist_test], y_test))

Epoch 1/8
3356/3356 [==============================] - 688s 203ms/step - loss: 6.5229 - accuracy: 0.0364 - val_loss: 6.2155 - val_accuracy: 0.0364
Epoch 2/8
3356/3356 [==============================] - 524s 156ms/step - loss: 6.1549 - accuracy: 0.0368 - val_loss: 6.2389 - val_accuracy: 0.0364
Epoch 3/8
3356/3356 [==============================] - 686s 205ms/step - loss: 6.1388 - accuracy: 0.0375 - val_loss: 6.2643 - val_accuracy: 0.0364
Epoch 4/8
3356/3356 [==============================] - 782s 233ms/step - loss: 6.1314 - accuracy: 0.0373 - val_loss: 6.2947 - val_accuracy: 0.0387
Epoch 5/8
3356/3356 [==============================] - 577s 172ms/step - loss: 6.1390 - accuracy: 0.0383 - val_loss: 6.3358 - val_accuracy: 0.0364
Epoch 6/8
3356/3356 [==============================] - 576s 172ms/step - loss: 6.1345 - accuracy: 0.0374 - val_loss: 6.3357 - val_accuracy: 0.0387
Epoch 7/8
3356/3356 [==============================] - 657s 196ms/step - loss: 6.1236 - accuracy: 0.0376 - val_loss: 6

In [76]:
def generate_lyrics(prompt, author, length = 20):
    '''
    prompt: string of lyrics
    length: length of lyrics that is wanted (includes prompt)
    '''
    # edge case; if prompt is as long as the length wanted
    if len(prompt.split(' ')) == length:
        return prompt
    elif subset_punk_df.ARTIST_NAME.str.contains(author).any() == False:
        return print(f'Artist not found! Try one of the following artists.\n{np.unique(subset_punk_df.ARTIST_NAME)}')
    else:
        a = [re.sub(r'\W', '', string = author)]
        
        a = np.array(tokenizer_artist.texts_to_sequences(a))
        
        for _ in range(20 - len(prompt.split(' '))):

            token_list = tokenizer_lyrics.texts_to_sequences([prompt])[0]
            token_padded = pad_sequences([token_list], maxlen = 10)

            # get predicted probability for each word
            predicted_probs = model.predict([token_padded, a])[0]

            # using temperature for next word
            probabilities = np.exp(predicted_probs / 1)
            normalized_probablities = probabilities / sum(probabilities)
            next_word = np.random.choice(range(vocab_size), p=normalized_probablities)
            next_word = tokenizer_lyrics.index_word[next_word]


            # add word to the prompt
            if len(next_word) > 1 and (next_word != 'a' or next_word != 'i'):
                prompt += ' ' + str(next_word)
        return prompt

In [80]:
oa = "there's a place off ocean avenue"

In [65]:
print(generate_lyrics("there's a place off ocean avenue", 'all time low', 20))

there's a place off ocean avenue mouldin' pace ups mais lifetime fads complication endeavor stitches fan overrated strangle wrong pairs


In [74]:
print(generate_lyrics("there's a place off ocean avenue", 'yellowcard', 20))

there's a place off ocean avenue unrestful bright snotty unkind sources given ambulances laundry ahold discard lays bags resolve rewind


In [77]:
print(generate_lyrics("there's a place off ocean avenue", '5 seconds of summer', 20))

Artist not found! Try one of the following artists.
['all time low' 'dashboard confessional' 'day to remember, a'
 'good charlotte' 'green day' 'jimmy eat world' 'menzingers, the'
 'my chemical romance' 'paramore' 'simple plan' 'state champs'
 'story of the year' 'story so far, the' 'taking back sunday' 'yellowcard']
None


In [79]:
generate_lyrics('my girlfriend likes my best friend', 'day to remember, a', 20)

"my girlfriend likes my best friend swimming whippin' hold vapors booze clause rock'n'rollers solved juliet degenerating used adjust swell cliche's"

In [81]:
generate_lyrics(oa, 'day to remember, a')

"there's a place off ocean avenue brightest killjoys waitress shy brown used bills fall motor girl's underdog highways collecting alcohol"

In [82]:
generate_lyrics(oa, 'my chemical romance')

"there's a place off ocean avenue negative razor inroads disposable satellite others questioned apology dorms teenage planned pour teaching stones"